## Create toy dataset

In [45]:
import torch
from pathlib import Path
import random

In [46]:
def generate_sphere_volume(
    shape=(64, 64, 64),
    radius_range=(4, 8),
):
    """
    Returns:
        volume: (1, D, H, W)
        box: (6,) -> (cx, cy, cz, dx, dy, dz)
    """
    D, H, W = shape
    volume = torch.zeros(1, D, H, W)

    r = random.randint(*radius_range)
    cx = random.randint(r, W - r - 1)
    cy = random.randint(r, H - r - 1)
    cz = random.randint(r, D - r - 1)

    z, y, x = torch.meshgrid(
        torch.arange(D),
        torch.arange(H),
        torch.arange(W),
        indexing="ij",
    )

    mask = (x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2 <= r ** 2
    volume[0][mask] = 1.0

    # Bounding box in (cx, cy, cz, dx, dy, dz)
    x1, x2 = cx - r, cx + r
    y1, y2 = cy - r, cy + r
    z1, z2 = cz - r, cz + r

    box = torch.tensor(
        [
            (x1 + x2) / 2,
            (y1 + y2) / 2,
            (z1 + z2) / 2,
            x2 - x1,
            y2 - y1,
            z2 - z1,
        ],
        dtype=torch.float32,
    )

    return volume, box

def create_toy_detection_dataset(
    root_dir,
    num_train=500,
    num_val=100,
    shape=(64, 64, 64),
):
    root = Path(root_dir)

    for split, n_samples in [("train", num_train), ("val", num_val)]:
        vol_dir = root / split / "volumes"
        tgt_dir = root / split / "targets"
        vol_dir.mkdir(parents=True, exist_ok=True)
        tgt_dir.mkdir(parents=True, exist_ok=True)

        for i in range(n_samples):
            volume, box = generate_sphere_volume(shape)

            torch.save(
                volume,
                vol_dir / f"sample_{i:04d}.pt",
            )

            torch.save(
                {
                    "boxes": box.unsqueeze(0),      # (1, 6)
                    "labels": torch.tensor([0]),    # single class
                },
                tgt_dir / f"sample_{i:04d}.pt",
            )

    print(f"Dataset written to: {root}")



In [47]:
create_toy_detection_dataset(
    root_dir="/home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset",
    num_train=1000,
    num_val=200,
    shape=(128,)*3
)

Dataset written to: /home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset


In [33]:
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

def visualize_3d_bbox(
    root_dir,
    split="train",
    sample_name=None,
):
    """
    Interactive visualization for a 3D volume with 3D bounding box.
    
    Expected structure:
    root_dir/
        train/
            volumes/sample_xxxx.pt
            targets/sample_xxxx.pt
        val/
            volumes/sample_xxxx.pt
            targets/sample_xxxx.pt

    target format:
    {
        'boxes': tensor([[x, y, z, dx, dy, dz]]),
        'labels': tensor([class_id])
    }
    """

    root_dir = Path(root_dir)
    vol_dir = root_dir / split / "volumes"
    tgt_dir = root_dir / split / "targets"

    samples = sorted([p.stem for p in vol_dir.glob("*.pt")])
    if len(samples) == 0:
        raise RuntimeError("No samples found")

    if sample_name is None:
        sample_name = samples[0]

    vol = torch.load(vol_dir / f"{sample_name}.pt")
    tgt = torch.load(tgt_dir / f"{sample_name}.pt")

    vol = vol.squeeze().cpu()
    box = tgt["boxes"][0].cpu()  # [x, y, z, dx, dy, dz]

    x, y, z, dx, dy, dz = box
    x0, x1 = int(x - dx / 2), int(x + dx / 2)
    y0, y1 = int(y - dy / 2), int(y + dy / 2)
    z0, z1 = int(z - dz / 2), int(z + dz / 2)

    x0, y0, z0 = max(x0, 0), max(y0, 0), max(z0, 0)
    x1, y1, z1 = min(x1, vol.shape[2]-1), min(y1, vol.shape[1]-1), min(z1, vol.shape[0]-1)

    def plot_slice(axis, idx):
        plt.figure(figsize=(5, 5))

        if axis == "z":
            img = vol[idx]
            if z0 <= idx <= z1:
                plt.gca().add_patch(
                    plt.Rectangle(
                        (x0, y0),
                        x1 - x0,
                        y1 - y0,
                        fill=False,
                        edgecolor="red",
                        linewidth=2,
                    )
                )
            plt.imshow(img, cmap="gray")

        elif axis == "y":
            img = vol[:, idx, :]
            if y0 <= idx <= y1:
                plt.gca().add_patch(
                    plt.Rectangle(
                        (x0, z0),
                        x1 - x0,
                        z1 - z0,
                        fill=False,
                        edgecolor="red",
                        linewidth=2,
                    )
                )
            plt.imshow(img, cmap="gray")

        elif axis == "x":
            img = vol[:, :, idx]
            if x0 <= idx <= x1:
                plt.gca().add_patch(
                    plt.Rectangle(
                        (y0, z0),
                        y1 - y0,
                        z1 - z0,
                        fill=False,
                        edgecolor="red",
                        linewidth=2,
                    )
                )
            plt.imshow(img, cmap="gray")

        plt.title(f"{sample_name} | axis={axis} | slice={idx}")
        plt.axis("off")
        plt.show()

    axis_dd = widgets.Dropdown(
        options=["z", "y", "x"],
        value="z",
        description="Axis:",
    )

    slice_slider = widgets.IntSlider(
        min=0,
        max=vol.shape[0] - 1,
        step=1,
        value=int(z),
        description="Slice:",
        continuous_update=False,
    )

    def update_slider(*args):
        axis = axis_dd.value
        if axis == "z":
            slice_slider.max = vol.shape[0] - 1
            slice_slider.value = int(z)
        elif axis == "y":
            slice_slider.max = vol.shape[1] - 1
            slice_slider.value = int(y)
        elif axis == "x":
            slice_slider.max = vol.shape[2] - 1
            slice_slider.value = int(x)

    axis_dd.observe(update_slider, names="value")

    ui = widgets.VBox([axis_dd, slice_slider])
    out = widgets.interactive_output(
        plot_slice,
        {"axis": axis_dd, "idx": slice_slider},
    )

    display(ui, out)


# Example usage in notebook:
# visualize_3d_bbox(
#     root_dir="/path/to/dataset",
#     split="train",
#     sample_name="sample_0001"
# )


In [36]:
visualize_3d_bbox(root_dir="/home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset", split="train", sample_name="sample_0011")

Output()

## Load the toy sphere dataset

In [1]:
from torch import nn
from qct_3d_nod_detect.structures import Instances3D, Boxes3D, ImagesList3D
import lightning.pytorch as pl
import torch
from typing import Optional, List, Dict

class GeneralizedRCNN3D(nn.Module):

    def __init__(
            self,
            backbone: nn.Module,
            rpn: nn.Module,
            roi_heads: nn.Module,
    ):

        super().__init__()
        self.backbone = backbone
        self.rpn = rpn
        self.roi_heads = roi_heads

    def forward(
            self,
            images: torch.Tensor,
            targets: Optional[List[Instances3D]] = None,
    ):
        
        """
        Returns:
            training: dict of losses
            inference: List[Instances3D]
        """

        features: Dict[str, torch.Tensor] = self.backbone(images)
        print(f"Feature extraction completed, FPN meta - ")
        for k, v in features.items():
            print(f"{k} - {v.shape}")

        image_list = ImagesList3D(
                tensor=images,
                image_sizes=[images.shape[-3:]] * images.shape[0],
                )
            
        proposals, rpn_losses = self.rpn(
            images=image_list,
            features=features,
            gt_instances=targets
        )

        roi_outputs = self.roi_heads(
            features=features,
            proposals=proposals,
            targets=targets
        )

        if self.training:
            losses = {}
            losses.update(rpn_losses)
            losses.update(roi_outputs)

            return losses
        
        else:
            return roi_outputs

def build_targets(batch):
    targets = []

    B = len(batch["gt_boxes"])
    image_size = batch["image"].shape[-3:]

    for i in range(B):
        inst = Instances3D(image_size=image_size)
        inst.gt_boxes = Boxes3D(batch["gt_boxes"][i])
        inst.gt_classes = batch["gt_classes"][i].long()
        targets.append(inst)

    return targets

def build_image_list_3d(images: torch.Tensor) -> ImagesList3D:
    """
    Args:
        images: Tensor[B, C, D, H, W]
    """
    image_sizes = [tuple(images.shape[-3:]) for _ in range(images.shape[0])]
    return ImagesList3D(image_sizes)

def build_instances_3d(batch):
    instances = []

    for boxes, classes in zip(batch["gt_boxes"], batch["gt_classes"]):
        inst = Instances3D(image_size=batch["image"].shape[-3:])
        inst.gt_boxes = Boxes3D(boxes)
        inst.gt_classes = classes
        instances.append(inst)

    return instances

# Lightning module
import torch
import lightning.pytorch as pl
from typing import List
from qct_3d_nod_detect.structures import Instances3D, Boxes3D, ImagesList3D
from qct_3d_nod_detect.base_lightning import BaseLightningModule

class FasterRCNN3DLightning(BaseLightningModule):

    def __init__(
            self,
            model: nn.Module,
            learning_rate: float = 1e-4,
            grad_clip_val: float = 0.0,
            grad_clip_algorithm: str = "norm",
            log_on: str = "step"
    ):

        super().__init__(learning_rate=learning_rate)

        self.model = model
        self.grad_clip_val = grad_clip_val
        self.grad_clip_algorithm = grad_clip_algorithm
        self.log_on = log_on
        
    def _build_targets(
        self,
        batch
    ):
        
        targets = []
        image_size = batch['image'].shape[-3:] # (D, H, W)

        for gt_boxes, gt_classes in zip(
            batch['gt_boxes'], batch['gt_classes']
        ):
            
            inst = Instances3D(image_size=image_size)
            inst.gt_boxes = Boxes3D(gt_boxes)
            inst.gt_classes = gt_classes.long()
            targets.append(inst)

        return targets
    
    def forward(
            self,
            images,
            targets=None,
    ):
        
        return self.model(images, targets)
    
    def training_step(
            self, 
            batch, 
            batch_idx
        ):

        images = batch["image"]
        targets = self._build_targets(batch)

        loss_dict: Dict[str, torch.Tensor] = self.model(images, targets)
        total_loss = sum(loss_dict.values())

        self.log_dict_helper(loss_dict, prefix="train/")
        
        self.log(
            "train/loss_total",
            total_loss,
            prog_bar=True,
            sync_dist=True
        )

        return total_loss
    
    def validation_step(
            self, 
            batch, 
            batch_idx
        ):

        images = batch["image"]
        targets = self._build_targets(batch)

        loss_dict = self.model(images, targets)
        total_loss = sum(loss_dict.values())

        self.log_dict_helper(loss_dict, prefix="val/")
        self.log(
            "val/loss_total",
            total_loss,
            prog_bar=True,
            sync_dist=True,
        )

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.learning_rate,
            weight_decay=1e-4,
        )

        return optimizer


In [22]:
# Dataset
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torch

class ToySphereDetectionDataset(Dataset):
    def __init__(self, root_dir, split="train"):
        self.root = Path(root_dir) / split
        self.vol_dir = self.root / "volumes"
        self.tgt_dir = self.root / "targets"

        self.ids = sorted(p.stem for p in self.vol_dir.glob("*.pt"))

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        sid = self.ids[idx]

        volume = torch.load(self.vol_dir / f"{sid}.pt")   # (1, D, H, W)
        target = torch.load(self.tgt_dir / f"{sid}.pt")

        return {
            "image": volume,                  # Tensor[1, D, H, W]
            "gt_boxes": target["boxes"],      # Tensor[N, 6]
            "gt_classes": target["labels"],   # Tensor[N]
        }
    
class ToySphereDetectionDataModule(pl.LightningDataModule):
    def __init__(
        self,
        root_dir: str,
        batch_size: int = 2,
        num_workers: int = 4,
        pin_memory: bool = True,
    ):
        super().__init__()
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory

    def setup(self, stage=None):
        # Called once per process
        self.train_dataset = ToySphereDetectionDataset(
            root_dir=self.root_dir,
            split="train",
        )

        self.val_dataset = ToySphereDetectionDataset(
            root_dir=self.root_dir,
            split="val",
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
            collate_fn=detection_collate,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
            collate_fn=detection_collate,
        )

def detection_collate(batch):
    return {
        "image": torch.stack([b["image"] for b in batch], dim=0),
        "gt_boxes": [b["gt_boxes"] for b in batch],
        "gt_classes": [b["gt_classes"] for b in batch],
    }

train_dataset = ToySphereDetectionDataset(
    root_dir="/home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset",
    split="train",
)

val_dataset = ToySphereDetectionDataset(
    root_dir="/home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset",
    split="val",
)

datamodule = ToySphereDetectionDataModule(root_dir='/home/users/ishan.tiwari/Ishan_Nodseg/qct_3d_nod_detect/toy_dataset',
                                          batch_size=2,
                                          num_workers=0,
                                          pin_memory=False)

In [23]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,          # start small
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    collate_fn=detection_collate,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=detection_collate,
)


In [24]:
batch = next(iter(train_dataloader))

print(batch["image"].shape)       # (B, 1, D, H, W)
print(len(batch["gt_boxes"]))     # B
print(batch["gt_boxes"][0].shape) # (N, 6)

torch.Size([2, 1, 128, 128, 128])
2
torch.Size([1, 6])


In [25]:
from qct_3d_nod_detect.rpn import RPN3D, StandardRPNHead3d
from qct_3d_nod_detect.faster_rcnn import FasterRCNNOutputLayers3D
from qct_3d_nod_detect.poolers import ROIPooler3D
from qct_3d_nod_detect.anchor_generator_3d import DefaultAnchorGenerator3D
from qct_3d_nod_detect.box_regression import Box3DTransform
from qct_3d_nod_detect.matcher import Matcher
from qct_3d_nod_detect.roi_heads import ROIHeads3D
from qct_3d_nod_detect.backbones import build_vit_backbone_with_fpn
from qct_3d_nod_detect.box_heads import FastRCNNConvFCHead3D
from qct_3d_nod_detect.layers import ShapeSpec
import math

In [26]:
anchor_generator_3d = DefaultAnchorGenerator3D(
    sizes=[[8], [16], [32], [64]],
    aspect_ratios_3d=[[(1.0, 1.0)], [(1.0, 1.0)], [(1.0, 1.0)], [(1.0, 1.0)]],
    strides=[4, 8, 16, 32],
    offset=0.5,
)

backbone_fpn = build_vit_backbone_with_fpn(
    variant="L",
    ckpt_path="/raid15/utkarsh.singh/ctssl/experiments/mae_vit3d_l_grid384x512x512_cs128_ps8_randmask09/checkpoints/best.ckpt",
    scales=[1, 2, 0.5, 0.25],
    out_channels=256
)

box3d2box3d_transform = Box3DTransform(
    weights=(1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
    scale_clamp=math.log(1000.0),
)

rpn_head_3d = StandardRPNHead3d(
    in_channels=256,
    num_anchors=anchor_generator_3d.num_cell_anchors[0],
    box_dim=6
)

rpn_matcher = Matcher(
    thresholds=[0.3, 0.7],
    labels=[0, -1, 1],
    allow_low_quality_matches=True,
)

roi_matcher = Matcher(
    thresholds=[0.5],
    labels=[0, 1],
    allow_low_quality_matches=False,
)

roi_pooler = ROIPooler3D(
    output_size=(7, 7, 7),
    canonical_level=4,
    canonical_box_size=224,
    pooler_type="ROIALign3DV2",
    scales=[1, 2, 0.5, 0.25]
)

rpn = RPN3D(
    in_features=["p2", "p3", "p4", "p5"],
    head=rpn_head_3d,
    anchor_generator=anchor_generator_3d,
    anchor_matcher=rpn_matcher,
    box3d_transform=box3d2box3d_transform,
    batch_size_per_image=256,
    positive_fraction=0.5,
    pre_nms_topk=(1000, 1000),
    post_nms_topk=(600, 600),
    nms_thresh=0.5,
    min_box_size=2.0,
    box_reg_loss_type="smooth_l1",
    smooth_l1_beta=0.0,
)

box_head = FastRCNNConvFCHead3D(
    input_shape=ShapeSpec(256, 7, 7, 7),
    conv_dims=[256,256],
    fc_dims=[512]
)

output_layers = FasterRCNNOutputLayers3D(
    input_dim=512,
    num_classes=1,
    box2box_transform=box3d2box3d_transform,
    cls_agnostic_bbox_reg=False,
)

roi_head = ROIHeads3D(
    num_classes=1,
    batch_size_per_image=10,
    positive_fraction=0.5,
    proposal_matcher=roi_matcher,
    roi_pooler=roi_pooler,
    proposal_append_gt=True,
    box_head=box_head,
    box_predictor=output_layers)

2026-01-22 08:26:21.804 | INFO     | qct_3d_nod_detect.backbones:load_state_dict_into_vit:351 - Missing MAE keys during load: %s


Missing keys: ['pos_embed'], Unexpected keys: []


In [27]:
model = GeneralizedRCNN3D(backbone=backbone_fpn, rpn=rpn, roi_heads=roi_head)
lit_model = FasterRCNN3DLightning(model=model, learning_rate=0.0001)

/home/users/ishan.tiwari/miniconda3/envs/qct_nod_seg/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [8]:
out = lit_model.training_step(batch=batch, batch_idx=0)

Feature extraction completed, FPN meta - 
p2 - torch.Size([2, 256, 32, 32, 32])
p3 - torch.Size([2, 256, 16, 16, 16])
p4 - torch.Size([2, 256, 8, 8, 8])
p5 - torch.Size([2, 256, 4, 4, 4])
Box features pool - torch.Size([20, 256, 7, 7, 7])
Box features shape - torch.Size([20, 512])


/home/users/ishan.tiwari/miniconda3/envs/qct_nod_seg/lib/python3.11/site-packages/lightning/pytorch/core/module.py:449: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(nan, grad_fn=<AddBackward0>)

In [ ]:
trainer = pl.Trainer(
    max_epochs=5,
    accelerator="gpu",        # or "cpu"
    devices=1,
    precision=32,             # start without AMP
    log_every_n_steps=5,
    enable_checkpointing=False,
    enable_model_summary=True,
)

trainer.fit(
    lit_model,
    datamodule=datamodule
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | GeneralizedRCNN3D | 399 M  | train
----------------------------------------------------
394 M     Trainable params
4.2 M     Non-trainable params
399 M     Total params
1,596.187 Total estimated model params size (MB)
445       Modules in train mode
0         Modules in eval mode
